In [31]:
!pip install simpy
import simpy
import random
import math
import numpy as np

In [32]:
Total_Time = 10

RANDOM_SEED = 1
random.seed(RANDOM_SEED)

hospital_treating_time_rate = 6
home_treating_time_rate = 10

num_of_individuals = 0
S = 147 + 177 + 207
if(S > 1000): num_of_individuals = S
elif(S > 10): num_of_individuals = S + 1000
else: num_of_individuals = S*300
    
total_sick = 0

num_of_individuals = 24
    
num_of_beds = math.ceil(num_of_individuals/12)
exponential_rate = num_of_individuals/300

print(num_of_individuals, exponential_rate, num_of_beds)

24 0.08 2


Define the necessary set of arrays for bookkeeping

In [33]:
service_times = [] #Duration of the conversation between the customer and the operator (Service time)

In [34]:
class Patient(object):
    def __init__(self, name, env, opr, isInitial):
        self.env = env
        self.name = name
        self.arrival_t = self.env.now
        self.r = random.uniform(1,2)
        self.hospital2home_treating_time_rate = 6*self.r
        self.isInitial = isInitial
        self.action = env.process(self.try_to_heal())
    
    
    def try_to_heal(self):
        if(not self.isInitial and self.is_going_hospital() == False):
            print('%s at home at %g' % (self.name, math.ceil(self.env.now)))
            yield self.env.process(self.healing())
            print('%s is healed at %g in home' % (self.name, math.ceil(self.env.now)))
        else:
            print('%s arrives hospital at %g' % (self.name, math.ceil(self.env.now)))
            with bed.request() as req:
                yield req
                print('%s is assigned to a bed at %g' % (self.name, math.ceil(self.env.now)))
                yield self.env.process(self.healing())
                print('%s is healed at %g' % (self.name, math.ceil(self.env.now)))
            
            
    def healing(self):
        if((self.is_going_hospital and bed.count != num_of_beds) or self.isInitial):
            duration = random.expovariate(1.0/hospital_treating_time_rate)
        elif(self.is_going_hospital):
            print("%s returns home." %(self.name))
            duration = random.expovariate(1.0/self.hospital2home_treating_time_rate)
        else:
            duration = random.expovariate(1.0/home_treating_time_rate)
        yield self.env.timeout(duration)
        service_times.append(math.ceil(duration))
        
        
    def is_going_hospital(self):
        random_number = random.random()
        if(random_number > 0.2): return False
        else: return True
    

In [35]:
def patient_generator(env, bed):
    """Generate new patients with using exponential rate!!!."""
    for i in range(Total_Time):
        sick_num_in_day = math.ceil(random.expovariate(1.0/exponential_rate))
        print('at day %s , %s people got sick' %((i+1), sick_num_in_day))
        
        #yield env.timeout(random.expovariate(exponential_rate))
        
        for j in range(sick_num_in_day):
            patient = Patient('Patient %s' %(total_sick+1), env, bed, False)
            total_sick = total_sick + 1
        
        yield env.timeout(1)

In [36]:
def patient_generator_half_full(env, bed):
    for i in range (num_of_beds // 2):
        yield env.timeout(0)
        patient = Patient('Patient initial %s' %(i+1), env, bed, True)
    for i in range(Total_Time):
        sick_num_in_day = math.ceil(random.expovariate(1.0/exponential_rate))
        print('at day %s , %s people got sick' %((i+1), sick_num_in_day))
        
        #yield env.timeout(random.expovariate(exponential_rate))
        
        for j in range(sick_num_in_day):
            patient = Patient('Patient %s' %(total_sick+1), env, bed, False)
            total_sick = total_sick + 1
        
        yield env.timeout(1)

In [37]:
def patient_generator_full(env, bed, total_sick):
    for i in range (num_of_beds):
        yield env.timeout(0)
        patient = Patient('Patient initial %s' %(i+1), env, bed, True)
    for i in range(Total_Time):
        sick_num_in_day = math.ceil(random.expovariate(1.0/exponential_rate))
        print('at day %s , %s people got sick' %((i+1), sick_num_in_day))
        
        #yield env.timeout(random.expovariate(exponential_rate))
        
        for j in range(sick_num_in_day):
            patient = Patient('Patient %s' %(total_sick+1), env, bed, False)
            total_sick = total_sick + 1
        
        yield env.timeout(1)

In [38]:
env = simpy.Environment()
bed = simpy.Resource(env, capacity = num_of_beds)
env.process(patient_generator_full(env, bed, total_sick))
env.run(until = Total_Time)

Patient initial 1 arrives hospital at 0
at day 1 , 1 people get sicked
Patient initial 2 arrives hospital at 0
Patient 1 at home at 0
Patient 1 returns home.
Patient initial 1 is assigned to a bed at 0
Patient initial 2 is assigned to a bed at 0
at day 2 , 1 people get sicked
Patient 2 at home at 1
Patient 2 returns home.
at day 3 , 1 people get sicked
Patient 3 at home at 2
Patient 3 returns home.
at day 4 , 1 people get sicked
Patient 4 at home at 3
Patient 4 returns home.
Patient 4 is healed at 4 in home
at day 5 , 1 people get sicked
Patient 5 at home at 4
Patient 5 returns home.
Patient 1 is healed at 5 in home
Patient 2 is healed at 5 in home
at day 6 , 1 people get sicked
Patient 6 arrives hospital at 5
at day 7 , 1 people get sicked
Patient 7 at home at 6
Patient 7 returns home.
Patient initial 1 is healed at 7
Patient 6 is assigned to a bed at 7
Patient 6 returns home.
at day 8 , 1 people get sicked
Patient 8 at home at 7
Patient 8 returns home.
Patient 8 is healed at 8 in hom

In [39]:
print(service_times)

[1, 5, 4, 7, 1, 3, 5, 3, 10, 8, 2]
